## Constant Volatility Fee Calculations

# ![Uniswap v3 fee tier formula](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*yah6u1x9uQkC7llZrz9QhQ.png)


In [3]:
import math

In [4]:
def getFeeTier(IV_per_year, tick_tvl, amount0, deltaT_secs):
 feeTier = IV_per_year / (2 * math.sqrt(365 * 24 * 60 * 60)) * math.sqrt(tick_tvl / amount0) * math.sqrt(deltaT_secs)
 return feeTier

In [5]:
# Example 1: 1 ETH trade with 100% annual volatility target
IV_per_year = 1.0  # 100% annual volatility - 10,000 bps, 1_000_000 pmds (aka 100th of a bip)
tick_tvl = 315     # Example TVL at tick
amount0 = 1        # 1 ETH trade
deltaT_secs = 15   # 1 block (15 seconds)

fee1 = getFeeTier(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 1 block: {fee1:.4%}")
# > Fee for 1 ETH trade after 1 block: 0.6120%

# Example 2: After 2 blocks
deltaT_secs = 30   # 2 blocks
fee2 = getFeeTier(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 2 blocks: {fee2:.4%}")
# > Fee for 1 ETH trade after 2 blocks: 0.8655%


# Example 3: 1000 ETH whale trade
amount0 = 1000     # 1000 ETH trade
deltaT_secs = 15   # 1 block
fee3 = getFeeTier(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1000 ETH whale trade: {fee3:.4%}")
# Fee for 1000 ETH whale trade: 0.0194%

# Total fees collected by LPs from whale trade
total_fees = amount0 * fee3
print(f"Total fees collected from whale trade: {total_fees:.2f} ETH")
# Total fees collected from whale trade: 0.19 ETH

Fee for 1 ETH trade after 1 block: 0.6120%
Fee for 1 ETH trade after 2 blocks: 0.8655%
Fee for 1000 ETH whale trade: 0.0194%
Total fees collected from whale trade: 0.19 ETH


In [ ]:
# TODO: calc fee in uniswap fee units, not %. then convert to pct after and log both